In [ ]:
import pandas as pd
df = pd.read_csv("../input/whale-categorization-playground/train.csv")

In [ ]:
df.head()

In [ ]:
from PIL import Image
import random

#show sample image
Image.open("../input/whale-categorization-playground/train/"+random.choice(df['Image'])) 

In [ ]:
from keras.preprocessing import image

In [ ]:
from keras.applications.imagenet_utils import preprocess_input

In [ ]:
# Prepare the images to be used for the training.  
# It changes their shape and converts it into an array.

def prepareImages(data, m, dataset):
    
    print("Preparing images")
    
    X_train = np.zeros((m, 100, 100, 3))
    
    count = 0
    
    for fig in data['Image']:
        
        #load images into images of size 100x100x3
        
        img = image.load_img("../input/whale-categorization-playground/"+dataset+"/"+fig, target_size=(100, 100, 3))
        x = image.img_to_array(img)
        
        
        x = preprocess_input(x)
        
        X_train[count] = x
        
        if (count%500 == 0):
            print("Processing image: ", count+1, ", ", fig)
        count += 1
    
    count = 0
    
    print("Finished!")
            
    return X_train

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from numpy import argmax

In [ ]:
from numpy import array

In [ ]:
# Preparing the labels, by converting them into one-hot vectors.

def prepareY(Y):

    values = array(Y)
    print(values.shape)
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)
    print(integer_encoded)

    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    print(onehot_encoded)

    y = onehot_encoded
    print(y.shape)
    return y, label_encoder

#the next lines are used to test the code and do not need to run when using it
#inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
#print(inverted)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, BatchNormalization, Activation, MaxPooling2D, AveragePooling2D, Dropout

In [ ]:
mod = Sequential()

mod.add(Conv2D(32, (7, 7), strides = (1, 1), name = 'conv0', input_shape = (100, 100, 3)))

mod.add(BatchNormalization(axis = 3, name = 'bn0'))
mod.add(Activation('relu'))

mod.add(MaxPooling2D((2, 2), name='max_pool'))
mod.add(Conv2D(64, (3, 3), strides = (1,1), name="conv1"))
mod.add(Activation('relu'))
mod.add(AveragePooling2D((3, 3), name='avg_pool'))

mod.add(Flatten())
mod.add(Dense(500, activation="relu", name='rl'))
mod.add(Dropout(0.8))
mod.add(Dense(4251, activation='softmax', name='sm'))

print(mod.output_shape)

#opt = optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

mod.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [ ]:
# Preparing data for training

X = prepareImages(df, 9850, "train")

#put all the values of the training data in the range between 0 and 1

X /= 255

print("Shape X-train: ", X.shape)

In [ ]:
Y = df['Id']

print("Shape Y-train: ", Y.shape)

In [ ]:
y, label_encoder = prepareY(Y)

In [ ]:
history = mod.fit(X, y, epochs=100, batch_size=100, verbose=1)

In [ ]:
import gc
gc.collect()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
#plot how the accuracy changes as the model was trained
plt.plot(history.history['accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.show()